In [ ]:
import importlib
import requests
from PIL import Image
import imghdr
import os 
import cv2
from io import BytesIO
import json
import numpy as np

# отправка запросов с видео файлом

In [ ]:
#файл, который отправляем
img = r'D:\projects\ero_video\dataset\insta\011.mp4' #без текста
img = r'D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\5b3012c99c38ea7955b363b9f1fb15bb805827a1.mp4' #видео с текстом
img = r'D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\7eef700e450d03ea7083a307636179958348f6bf.mp4' #видео с текстом ВЫДАЕТ ОШИБКУ

img = r'D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\12e8647a0c07e16045a912a8953297061e1e804e.mp4' 
img = r'D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом\a7ee7b99b46830da26d322e9f75e4a3c9f04aab0.mp4' 



url = "http://127.0.0.1:8000/vd"  #ЮРЛ - другой. Он отличается от картинки
file = {"video": open(img, "rb")}  #открываем файл как бинарный

response = requests.post(url, files=file)

print(response.headers)
if response.status_code == 200:
   
    result = response.json()
    print( result)

else:
    print(f"Error: {response.status_code} - {response.text}")



#ОПИСАНИЕ СТРУКТУРЫ ОТВЕТА
#json структура. 
#need_moderation
    #1 - нужна модерация
    #0 - модераци не нужна, текстов не найдено

# Если need_moderation = 1, то появится еще один ключ timeslots
# В нем в списке хранятся словари {'timesslot': [sN, sF], 'frame_data': [список целых чисел]}
# timesslot - начало и окончание кадров с текстом в видео. 
# frame_data - первый кадр, где начинается текст
# Таких словарей может быть несколько в зависимости от того, сколько найдено таймслотов. 

# Как из framme_data получать изображение:
# jpeg_bytes_list = record['frame_data'] #в обзем виде это массив чисел, который сейчас будем перепаковывать в jpg
# jpeg_bytes_array = np.array(jpeg_bytes_list, dtype=np.uint8)
# image = cv2.imdecode(jpeg_bytes_array, flags=cv2.IMREAD_COLOR)

# image - будет содержать картинку. Далее с ним можно делать всё, что хотим: сохранить, показать,  и т.д... далее есть пример


# ПОЛНЫЙ ПРИМЕР

In [ ]:
#перебираем все файлы, отправляем их на модерацию, получам и обрабатываем ответ, сохраняем полученные кадры с тектом

In [ ]:
#функция получает ответ API, и достает из jsona все jpg и таймсслоты
def unpack_jpg_from_json(rez,item=''):
    if rez['need_moderation']:
        print('нужна модерация')
        #достаем фото
        for record in rez['timeslots']:
            print(record['timesslot']) #это наш таймслот. их в ответе может быть от 1 до много
            jpeg_bytes_list = record['frame_data'] #в обзем виде это массив чисел, который сейчас будем перепаковывать в jpg
            jpeg_bytes_array = np.array(jpeg_bytes_list, dtype=np.uint8)
            image = cv2.imdecode(jpeg_bytes_array, flags=cv2.IMREAD_COLOR)
            #сохраняем сюда
            key_frame = record['timesslot'][0] #ключевой кадр
            cv2.imwrite(f'Y:\decoded_image {item} {key_frame}.jpg', image)
    else:
        print('Модерация пройдена')





In [ ]:
url = "http://127.0.0.1:8000/vd"  #ЮРЛ - другой. 
    
def get_moderation_video_file(video_file,item):
    file = {"video": open(video_file, "rb")}  #открываем файл как бинарный

    response = requests.post(url, files=file)
    if response.status_code == 200:
        result = response.json()
        #print(result)
        unpack_jpg_from_json(result,item)
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [ ]:
#Тестируем. Перебираем все файлы полученные
import os

directory_path = r'D:\projects\ero_txt\_VIDEO\dataset\test 30\Видео с текстом'
# directory_path = r'D:\projects\ero_txt\_VIDEO\dataset\test 30\видео без текста'
#directory_path =  r'D:\projects\ero_txt\_VIDEO\dataset\test 1000'

files_and_dirs = os.listdir(directory_path)

for item in files_and_dirs:
    work_file = os.path.join(directory_path, item)
    print(work_file)
    get_moderation_video_file(work_file,item)

# отправка запросов с изображением файлом

In [ ]:
img  = r'D:\projects\ero_txt\txt_girls\data_set\inp_2023-12-03 19-09-00.jpg' #пример с текстом
#img  = r'D:\projects\ero_txt\Оружие\2024-05-20_12-09-55.png'

url = "http://localhost:8000/ps"
file = {"image": open(img, "rb")}  #открываем файл как бинарный


response = requests.post(url, files=file)

#print(response.headers)
if response.status_code == 200:
    result = response.json()
    print( result)

#json структура. 
#need_moderation
    #1 - нужна модерация
    #0 - модераци не нужна, текстов не найдено